In [149]:
import compyute as cp

from compyute.functional import zeros_like
from compyute.nn.funcional import linear, linear_backward, sigmoid
from compyute.nn import Module
from compyute.nn.parameter import Parameter
from compyute.random import uniform
from compyute.tensor import Tensor
from compyute.types import DtypeLike

from compyute.nn.modules.layers import LSTMCell

from tests.test_utils import validate

In [150]:


# class LSTMCell(Module):
#     """Long Short-Term Memory cell."""

#     def __init__(
#         self,
#         in_channels: int,
#         h_channels: int,
#         bias: bool = True,
#         dtype: DtypeLike = "float32",
#     ) -> None:
#         """Long Short-Term Memory cell.
#         Input: (B, T, Cin)
#             B ... batch, T ... time, Cin ... input channels
#         Output: (B, Ch)
#             B ... batch, Ch ... hidden channels

#         Parameters
#         ----------
#         in_channels : int
#             Number of input features.
#         h_channels : int
#             Number of hidden channels.
#         bias : bool, optional
#             Whether to use bias values, by default True.
#         dtype: DtypeLike, optional
#             Datatype of weights and biases, by default "float32".
#         """
#         super().__init__()
#         self.in_channels = in_channels
#         self.h_channels = h_channels
#         self.bias = bias
#         self.dtype = dtype

#         k = in_channels**-0.5

#         # init if
#         w_if = uniform((h_channels, in_channels), -k, k)
#         self.w_if = Parameter(w_if, dtype=dtype, label="w_if")
#         self.b_if = (
#             Parameter(uniform((h_channels,), -k, k), dtype=dtype, label="b_if")
#             if bias
#             else None
#         )

#         # init hf
#         w_hf = uniform((h_channels, h_channels), -k, k)
#         self.w_hf = Parameter(w_hf, dtype=dtype, label="w_hf")
#         self.b_hf = (
#             Parameter(uniform((h_channels,), -k, k), dtype=dtype, label="b_hf")
#             if bias
#             else None
#         )

#         # init ii
#         w_ii = uniform((h_channels, in_channels), -k, k)
#         self.w_ii = Parameter(w_ii, dtype=dtype, label="w_ii")
#         self.b_ii = (
#             Parameter(uniform((h_channels,), -k, k), dtype=dtype, label="b_ii")
#             if bias
#             else None
#         )

#         # init hi
#         w_hi = uniform((h_channels, h_channels), -k, k)
#         self.w_hi = Parameter(w_hi, dtype=dtype, label="w_hi")
#         self.b_hi = (
#             Parameter(uniform((h_channels,), -k, k), dtype=dtype, label="b_hi")
#             if bias
#             else None
#         )

#         # init ig
#         w_ig = uniform((h_channels, in_channels), -k, k)
#         self.w_ig = Parameter(w_ig, dtype=dtype, label="w_ig")
#         self.b_ig = (
#             Parameter(uniform((h_channels,), -k, k), dtype=dtype, label="b_ig")
#             if bias
#             else None
#         )

#         # init hg
#         w_hg = uniform((h_channels, h_channels), -k, k)
#         self.w_hg = Parameter(w_hg, dtype=dtype, label="w_hg")
#         self.b_hg = (
#             Parameter(uniform((h_channels,), -k, k), dtype=dtype, label="b_hg")
#             if bias
#             else None
#         )

#         # init io
#         w_io = uniform((h_channels, in_channels), -k, k)
#         self.w_io = Parameter(w_io, dtype=dtype, label="w_io")
#         self.b_io = (
#             Parameter(uniform((h_channels,), -k, k), dtype=dtype, label="b_io")
#             if bias
#             else None
#         )

#         # init ho
#         w_ho = uniform((h_channels, h_channels), -k, k)
#         self.w_ho = Parameter(w_ho, dtype=dtype, label="w_ho")
#         self.b_ho = (
#             Parameter(uniform((h_channels,), -k, k), dtype=dtype, label="b_ho")
#             if bias
#             else None
#         )

#     def forward(self, x: Tensor):
#         self.check_dims(x, [3])
#         x = x.astype(self.dtype)

#         # input projections
#         # (B, T, Cin) @ (Cin, Ch) + (Ch,) -> (B, T, Ch)
#         f_h = linear(x, self.w_if, self.b_if)
#         i_h = linear(x, self.w_ii, self.b_ii)
#         g_h = linear(x, self.w_ig, self.b_ig)
#         o_h = linear(x, self.w_io, self.b_io)

#         # iterate over timesteps
#         # (B, T, Ch)
#         f = zeros_like(f_h)
#         i = zeros_like(i_h)
#         g = zeros_like(g_h)
#         o = zeros_like(o_h)
#         c = zeros_like(f_h)
#         h = zeros_like(f_h)

#         for t in range(x.shape[1]):
#             # (B, 4*Ch) + (B, Ch) @ (Ch, 4*Ch) + (4*Ch,) -> (B, 4*Ch)
#             f[:, t] = sigmoid(f_h[:, t] + linear(h[:, t - 1], self.w_hf, self.b_hf))
#             i[:, t] = sigmoid(i_h[:, t] + linear(h[:, t - 1], self.w_hi, self.b_hi))
#             g[:, t] = (g_h[:, t] + linear(h[:, t - 1], self.w_hg, self.b_hg)).tanh()
#             o[:, t] = sigmoid(o_h[:, t] + linear(h[:, t - 1], self.w_ho, self.b_ho))

#             c[:, t] = f[:, t] * c[:, t - 1] + i[:, t] * g[:, t]
#             h[:, t] = o[:, t] * c[:, t].tanh()

#         if self.training:

#             # https://pureai.com/articles/2019/11/14/~/media/ECG/pureai/Images/2019/11/lstm_1.asxh

#             def backward(dy: Tensor) -> Tensor:
#                 dh = dy.astype(self.dtype)
#                 self.set_dy(dh)

#                 df = zeros_like(f)
#                 dc = zeros_like(c)

#                 d_sig_f = zeros_like(f)
#                 d_sig_i = zeros_like(i)
#                 d_tanh_g = zeros_like(g)
#                 d_sig_o = zeros_like(o)

#                 self.w_hf.grad = zeros_like(self.w_hf)
#                 self.w_hi.grad = zeros_like(self.w_hi)
#                 self.w_hg.grad = zeros_like(self.w_hg)
#                 self.w_ho.grad = zeros_like(self.w_ho)

#                 for t in range(x.shape[1] - 1, -1, -1):
#                     out_grad = dh[:, t]
#                     if t < x.shape[1] - 1:
#                         out_grad += d_sig_f[:, t + 1] @ self.w_hf
#                         out_grad += d_sig_i[:, t + 1] @ self.w_hi
#                         out_grad += d_tanh_g[:, t + 1] @ self.w_hg
#                         out_grad += d_sig_o[:, t + 1] @ self.w_ho

#                     do_t = c[:, t].tanh() * out_grad

#                     dc[:, t] = (1 - c[:, t].tanh() ** 2) * o[:, t] * out_grad
#                     if t < x.shape[1] - 1:
#                         dc[:, t] += f[:, t + 1] * dc[:, t + 1]

#                     df_t = (c[:, t - 1] * dc[:, t]) if t > 0 else 0
#                     di_t = g[:, t] * dc[:, t]
#                     dg_t = i[:, t] * dc[:, t]

#                     d_sig_f[:, t] = f[:, t] * (1 - f[:, t]) * df_t
#                     d_sig_i[:, t] = i[:, t] * (1 - i[:, t]) * di_t
#                     d_tanh_g[:, t] = (1 - g[:, t] ** 2) * dg_t
#                     d_sig_o[:, t] = o[:, t] * (1 - o[:, t]) * do_t

#                     if t > 0:
#                         self.w_hf.grad += d_sig_f[:, t].T @ h[:, t - 1]
#                         self.w_hi.grad += d_sig_i[:, t].T @ h[:, t - 1]
#                         self.w_hg.grad += d_tanh_g[:, t].T @ h[:, t - 1]
#                         self.w_ho.grad += d_sig_o[:, t].T @ h[:, t - 1]

#                 self.b_hf.grad = d_sig_f.sum(axis=(0, 1))
#                 self.b_hi.grad = d_sig_i.sum(axis=(0, 1))
#                 self.b_hg.grad = d_tanh_g.sum(axis=(0, 1))
#                 self.b_ho.grad = d_sig_o.sum(axis=(0, 1))

#                 dx = linear_backward(d_sig_f, x, self.w_if, self.b_if)
#                 dx += linear_backward(d_sig_i, x, self.w_ii, self.b_ii)
#                 dx += linear_backward(d_tanh_g, x, self.w_ig, self.b_ig)
#                 dx += linear_backward(d_sig_o, x, self.w_io, self.b_io)

#                 return dx

#             self.backward = backward

#         self.set_y(h)
#         return h


In [151]:
N, L, Hin, Hout = 5, 3, 4, 6

X = cp.random.normal((N, L, Hin)).float()

In [152]:
lstm = LSTMCell(Hin, Hout)
lstm.training = True

In [153]:
import torch
import torch.nn as nn

torch_lstm = torch.nn.LSTM(Hin, Hout, batch_first=True)

wih_stack = cp.concatenate([lstm.w_ii, lstm.w_if, lstm.w_ig, lstm.w_io], axis=0)
torch_lstm.weight_ih_l0 = nn.Parameter(torch.Tensor(wih_stack.to_numpy()), requires_grad=True)

bih_stack = cp.concatenate([lstm.b_ii, lstm.b_if, lstm.b_ig, lstm.b_io], axis=0)
torch_lstm.bias_ih_l0 = nn.Parameter(torch.Tensor(bih_stack.to_numpy()), requires_grad=True)

whh_stack = cp.concatenate([lstm.w_hi, lstm.w_hf, lstm.w_hg, lstm.w_ho], axis=0)
torch_lstm.weight_hh_l0 = nn.Parameter(torch.Tensor(whh_stack.to_numpy()), requires_grad=True)

bhh_stack = cp.concatenate([lstm.b_hi, lstm.b_hf, lstm.b_hg, lstm.b_ho], axis=0)
torch_lstm.bias_hh_l0 = nn.Parameter(torch.Tensor(bhh_stack.to_numpy()), requires_grad=True)

In [154]:
out = lstm(X)
torch_X = torch.tensor(X.to_numpy(), requires_grad=True)
torch_out = torch_lstm(torch_X)[0]

validate(out, torch_out)

True

In [155]:
dy = cp.random.normal(out.shape)
x_grad = lstm.backward(dy)
torch_dy = torch.Tensor(dy.to_numpy())
torch_out.backward(torch_dy)


validate(x_grad, torch_X.grad)

True

In [156]:
validate(cp.concatenate([lstm.w_hi.grad, lstm.w_hf.grad, lstm.w_hg.grad, lstm.w_ho.grad], 0), torch_lstm.weight_hh_l0.grad)

True

In [157]:
validate(cp.concatenate([lstm.b_hi.grad, lstm.b_hf.grad, lstm.b_hg.grad, lstm.b_ho.grad], 0), torch_lstm.bias_hh_l0.grad)

True

In [158]:
validate(cp.concatenate([ lstm.w_ii.grad, lstm.w_if.grad, lstm.w_ig.grad, lstm.w_io.grad], 0), torch_lstm.weight_ih_l0.grad)

True

In [159]:
validate(cp.concatenate([lstm.b_ii.grad, lstm.b_if.grad, lstm.b_ig.grad, lstm.b_io.grad], 0), torch_lstm.bias_ih_l0.grad)

True